In [3]:
# 실행 후 런타임 > 세션 다시시작 후 아래셀부터 실행하기
!pip install -U -q datasets transformers fsspec
!pip install -q sentence-transformers faiss-cpu matplotlib seaborn pandas numpy tqdm scikit-learn nltk python-Levenshtein

1. **데이터셋 로드 및 준비**:
   - tatsu-lab/alpaca
   - yahma/alpaca-cleaned  
   - databricks/databricks-dolly-15k
   - 각 데이터셋의 구조 분석 및 통합 포맷으로 변환

2. **Synthetic Deduplication 구현**:
   - Exact Matching (해시 기반)
   - Normalized Matching (정규화 기반)
   - Edit Distance (Levenshtein 거리)

In [21]:
from datasets import load_dataset, DatasetDict
import pandas as pd
import hashlib
import re
import Levenshtein
from sentence_transformers import SentenceTransformer
import faiss
import time
import tracemalloc

# 데이터셋 로드 및 구조 확인
ds1 = load_dataset("tatsu-lab/alpaca")
ds2 = load_dataset("yahma/alpaca-cleaned")
ds3 = load_dataset("databricks/databricks-dolly-15k")
# ds4 = load_dataset("timdettmers/openassistant-guanaco")

# 데이터셋 개수 확인
print("tatsu-lab/alpaca:", len(ds1["train"]))
print("yahma/alpaca-cleaned:", len(ds2["train"]))
print("databricks/databricks-dolly-15k:", len(ds3["train"]))

# 예시: train split만 살펴보기
print("─ tatsu-lab/alpaca sample ─")
print(ds1["train"][0])
print("\n─ yahma/alpaca-cleaned sample ─")
print(ds2["train"][0])
print("\n─ databricks/databricks-dolly-15k sample ─")
print(ds3["train"][0])
# print("\n─ openassistant-guanaco sample ─")
# print(ds4["train"][0])


Repo card metadata block was not found. Setting CardData to empty.


tatsu-lab/alpaca: 52002
yahma/alpaca-cleaned: 51760
databricks/databricks-dolly-15k: 15011
─ tatsu-lab/alpaca sample ─
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

─ yahma/alpaca-cleaned sample ─
{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and health

In [8]:
# 공통 포맷 변환 (instruction, input, output)
def unify(ds, cols_map):
    df = pd.DataFrame(ds)
    df = df.rename(columns=cols_map)[list(cols_map.values())]
    return df

df1 = unify(ds1["train"], {"instruction": "instruction", "input": "input", "output": "output"})
df2 = unify(ds2["train"], {"instruction": "instruction", "input": "input", "output": "output"})
df3 = unify(ds3["train"], {"instruction": "instruction",  "context": "input", "response": "output"})
# df4 = unify(ds4["train"], {"instruction": "instruction", "input": "input",   "output": "response"})

df_all = pd.concat([df1, df2, df3, ], ignore_index=True)
print("통합 데이터셋 크기:", df_all.shape)
print(df_all.head(3))


통합 데이터셋 크기: (118773, 3)
                            instruction input  \
0  Give three tips for staying healthy.         
1    What are the three primary colors?         
2    Describe the structure of an atom.         

                                              output  
0  1.Eat a balanced diet and make sure to include...  
1  The three primary colors are red, blue, and ye...  
2  An atom is made up of a nucleus, which contain...  


In [33]:
# Synthetic Deduplication - Exact Matching (해시 기반)
def hash_text(row):
    return hashlib.md5((row.instruction + row.input + row.output).encode()).hexdigest()

df_all["hash"] = df_all.apply(hash_text, axis=1)
before = len(df_all)
df_exact = df_all.drop_duplicates("hash")
after = len(df_exact)
print(f"Exact Matching: 중복 제거 전 {before} → 제거 후 {after}")
print(f"제거 sample 수: {before - after}")
print(f"제거 sample 비율: {(before - after) / before * 100:.2f}%")

# 중복데이터 예시 출력
dups_norm = df_all[df_all.duplicated("norm", keep=False)]
groups_norm = dups_norm.groupby("norm")
print("\n🔍 Normalized Matching 중복 예시 (2개 그룹):")
for i, (key, group) in enumerate(groups_norm):
    if i >= 2:
        break
    print(f"\n그룹 {i+1}:")
    for _, row in group.head(2).iterrows():
        print("Instruction:", row.instruction)
        print("Input      :", row.input)
        print("Output     :", row.output)
        print("---")

Exact Matching: 중복 제거 전 118773 → 제거 후 116922
제거 sample 수: 1851
제거 sample 비율: 1.56%

🔍 Normalized Matching 중복 예시 (2개 그룹):

그룹 1:
Instruction: According to the given context, classify the following sentence into whether it is true or false
Input      : Context: Life can be unfair
Sentence: Life should always be fair
Output     : False
---
Instruction: According to the given context, classify the following sentence into whether it is true or false
Input      : Context: Life can be unfair
Sentence: Life should always be fair
Output     : False
---

그룹 2:
Instruction: According to the input sentence, create a list of words.
Input      : "The cat is black and white"
Output     : ["The", "cat", "is", "black", "and", "white"]
---
Instruction: According to the input sentence, create a list of words.
Input      : "The cat is black and white"
Output     : ["The", "cat", "is", "black", "and", "white"]
---


In [35]:
# Synthetic Deduplication - Normalized Matching (정규화)
def normalize(text):
    text = text.lower()
    text = re.sub(r"\W+", " ", text)
    return text.strip()

df_exact["norm"] = df_exact.instruction.map(normalize) + "|" + df_exact.input.map(normalize) + "|" + df_exact.output.map(normalize)
before = len(df_exact)
df_norm = df_exact.drop_duplicates("norm")
after = len(df_norm)
print(f"Normalized Matching: 중복 제거 전 {before} → 제거 후 {after}")
print(f"제거 sample 수: {before - after}")
print(f"제거 sample 비율: {(before - after) / before * 100:.2f}%")

# ↓ 제거된 예시 출력 ↓
# df_exact[df_exact.duplicated("norm", False)].sort_values("norm").head(10)

# 중복데이터 예시 출력
dups_norm = df_exact[df_exact.duplicated("norm", keep=False)]
groups_norm = dups_norm.groupby("norm")
print("\n🔍 Normalized Matching 중복 예시 (2개 그룹):")
for i, (key, group) in enumerate(groups_norm):
    if i >= 2:
        break
    print(f"\n그룹 {i+1}:")
    for _, row in group.head(2).iterrows():
        print("Instruction:", row.instruction)
        print("Input      :", row.input)
        print("Output     :", row.output)
        print("---")

Normalized Matching: 중복 제거 전 116922 → 제거 후 116200
제거 sample 수: 722
제거 sample 비율: 0.62%

🔍 Normalized Matching 중복 예시 (2개 그룹):

그룹 1:
Instruction: Add 10 words that start with the letter ‘M’ to the given list.
Input      : bank, car, tree
Output     : bank, car, tree, muffin, museum, melody, magnet, mascot, mop, machete, mattress
---
Instruction: Add 10 words that start with the letter "M" to the given list.
Input      : bank, car, tree
Output     : bank, car, tree, muffin, museum, melody, magnet, mascot, mop, machete, mattress
---

그룹 2:
Instruction: Add a comma in the appropriate place.
Input      : The dog chased the cat across the yard
Output     : The dog, chased the cat across the yard.
---
Instruction: Add a comma in the appropriate place.
Input      : The dog chased the cat across the yard
Output     : The dog chased the cat, across the yard.
---


/tmp/ipython-input-35-718881863.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exact["norm"] = df_exact.instruction.map(normalize) + "|" + df_exact.input.map(normalize) + "|" + df_exact.output.map(normalize)


In [38]:
len(df_exact[df_exact.duplicated("norm", keep=False)])

1444

In [43]:
from tqdm import tqdm

# Synthetic Deduplication - Edit Distance (Levenshtein)
def levenshtein_dedup_with_examples(df, threshold=5):
    keep = []
    dup_pairs = []
    for idx, row in tqdm(df.iterrows(),total=len(df)):
        text = row.instruction + row.input + row.output
        is_dup = False
        for j in keep:
            other = df.loc[j]
            dist = Levenshtein.distance(text, other.instruction + other.input + other.output)
            if dist <= threshold:
                is_dup = True
                dup_pairs.append((j, idx))
                break
        if not is_dup:
            keep.append(idx)
    return df.loc[keep], dup_pairs

# 가상의 중복데이터 생성
similar_df = pd.concat([df_exact[df_exact.duplicated("norm", keep=False)], df_exact.sample(100)])

start = time.time()
# df_edit = levenshtein_dedup_with_examples(df_norm_sample, threshold=20)
df_edit, dup_pairs = levenshtein_dedup_with_examples(similar_df, threshold=20)
elapsed = time.time() - start

before=len(similar_df)
after=len(df_edit)

print(f"Edit Distance Dedup: 최종 크기 {after}, 처리 시간 {elapsed:.2f}s")

print(f"Edit Distance Dedup: 최종 크기 {after}, 처리 시간 {elapsed:.2f}s")
print(f"Normalized Matching: 중복 제거 전 {before} → 제거 후 {after}")
print(f"제거 sample 수: {before - after}")
print(f"제거 sample 비율: {(before - after) / before * 100:.2f}%")

# 중복데이터 예시 출력

print("\n🔍 Levenshtein 중복 예시 (거리 포함):")
for i, (orig_idx, dup_idx) in enumerate(dup_pairs[:2]):
    orig = similar_df.loc[orig_idx]
    dup  = similar_df.loc[dup_idx]
    # 거리 계산
    dist = Levenshtein.distance(
        orig.instruction + orig.input + orig.output,
        dup.instruction  + dup.input  + dup.output
    )

    print(f"\n중복데이터 {i+1}: (Distance = {dist})")
    print("▶ 원본 Instruction:", orig.instruction)
    print("▶ Duplicate Instruction:", dup.instruction)
    print("▶ 원본 Input      :", orig.input)
    print("▶ Duplicate Input      :", dup.input)
    print("▶ 원본 Output     :", orig.output)
    print("▶ Duplicate Output     :", dup.output)
    print("---")

100%|██████████| 1544/1544 [00:59<00:00, 25.96it/s]

Edit Distance Dedup: 최종 크기 833, 처리 시간 59.47s
Edit Distance Dedup: 최종 크기 833, 처리 시간 59.47s
Normalized Matching: 중복 제거 전 1544 → 제거 후 833
제거 sample 수: 711
제거 sample 비율: 46.05%

🔍 Levenshtein 중복 예시 (거리 포함):

중복데이터 1: (Distance = 20)
▶ 원본 Instruction: What is the chemical symbol of iron?
▶ Duplicate Instruction: Give the chemical symbol for Carbon
▶ 원본 Input      : 
▶ Duplicate Input      : 
▶ 원본 Output     : The chemical symbol of iron is Fe.
▶ Duplicate Output     : The chemical symbol for Carbon is C.
---

중복데이터 2: (Distance = 12)
▶ 원본 Instruction: Tell us a joke
▶ Duplicate Instruction: Type out a short joke.
▶ 원본 Input      : 
▶ Duplicate Input      : 
▶ 원본 Output     : Why don't scientists trust atoms? Because they make up everything!
▶ Duplicate Output     : Why don't scientists trust atoms? Because they make up everything!
---


In [42]:
dist = Levenshtein.distance(
        "iron ","Carbon"
    )
dist

4